<a href="https://colab.research.google.com/github/Harish-15112003/MCW_simple_bev/blob/main/Simple_Bev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!git clone https://github.com/aharley/simple_bev.git
%cd simple_bev

!pip install torch==1.12.0 torchvision==0.13.0
!pip install -r requirements.txt


Cloning into 'simple_bev'...
remote: Enumerating objects: 375, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 375 (delta 42), reused 45 (delta 41), pack-reused 326 (from 1)
Receiving objects: 100% (375/375), 144.00 KiB | 9.00 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/content/simple_bev
ERROR: Could not find a version that satisfies the requirement torch==1.12.0 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.12.0
  Cloning https://github.com/lyft/nuscenes-devkit to /tmp/pip-req-build-prz7krz5
  Running command git clone --filter=blob:none --quiet https://github.com/lyft/nuscenes-devkit /tmp/pip-req-build-prz7krz5
  Resolved https://github.com/lyft/nuscenes-devkit to commit 49c36da0a85da6bc9e8f2a39d5d967311cd75069
  Installing build dependencies ... done
  Getting requirem

In [4]:
!bash get_rgb_model.sh
!bash get_rad_model.sh


downloading rgb model from dropbox...
--2025-03-09 06:02:37--  https://www.dropbox.com/s/n93ryvrqyiram56/rgb_checkpoint.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/k4pk5szq7i0gul2aqylb1/rgb_checkpoint.tar.gz?rlkey=6fmruxvsvh6nij3lfuyjn51ub [following]
--2025-03-09 06:02:37--  https://www.dropbox.com/scl/fi/k4pk5szq7i0gul2aqylb1/rgb_checkpoint.tar.gz?rlkey=6fmruxvsvh6nij3lfuyjn51ub
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc442ffcb4b75e4da643e3a6793.dl.dropboxusercontent.com/cd/0/inline/ClhQvO96IZ5hLYatHqzef3hSY5cYjG4PTHfRam9Ma0yarqS2hmCgkjpZO9vbmgXe01ShHe1B96mNBGUDXSInrW9J2JwJRuPpr_kpWe4O1KXfutGfPKhV-FvAAXp73u7lunA/file# [following]
--2025-03-09 06:02:37--  https://ucc442ffcb4b75e4da643e3a

In [5]:
pwd

'/content/simple_bev'

In [6]:
!mkdir -p /content/simple_bev/datasets/nuscenes  # Make the directory to store the nuScenes dataset in.

!wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.



--2025-03-09 06:03:17--  https://www.nuscenes.org/data/v1.0-mini.tgz
Resolving www.nuscenes.org (www.nuscenes.org)... 13.224.14.15, 13.224.14.105, 13.224.14.80, ...
Connecting to www.nuscenes.org (www.nuscenes.org)|13.224.14.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4167696325 (3.9G) [application/x-tar]
Saving to: ‘v1.0-mini.tgz’

v1.0-mini.tgz       100%[===================>]   3.88G  80.5MB/s    in 50s     

2025-03-09 06:04:07 (79.0 MB/s) - ‘v1.0-mini.tgz’ saved [4167696325/4167696325]



In [7]:
!tar -xf /content/simple_bev/v1.0-mini.tgz -C /content/simple_bev/datasets/nuscenes # Uncompress the nuScenes mini split.

!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.

In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)


cuda


In [9]:
!pip install tensorboardX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.0 MB/s eta 0:00:00


In [10]:
!pip install efficientnet_pytorch

  Using cached efficientnet_pytorch-0.7.1.tar.gz (21 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 973.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.3 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=164

In [11]:
!pip install lyft_dataset_sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00


In [16]:
!python eval_nuscenes.py \
    --batch_size=4 \
    --data_dir='/content/simple_bev/datasets/nuscenes' \
    --log_dir='logs_eval_nuscenes_bevseg' \
    --init_dir='/content/simple_bev/checkpoints/8x5_5e-4_rgb12_22:43:46' \
    --res_scale=2 \
    --device_ids=[0]


model_name 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32
resolution: (448, 800)
loading nuscenes...
making parser...
indices (323, 1)
NuscData: 323 samples. Split: train.
                   Augmentation Conf: {'final_dim': (448, 800), 'cams': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'], 'ncams': 6}
indices (81, 1)
NuscData: 81 samples. Split: val.
                   Augmentation Conf: {'final_dim': (448, 800), 'cams': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'], 'ncams': 6}
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/f

In [ ]:
# odel_name 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32
# resolution: (448, 800)
# loading nuscenes...
# making parser...
# indices (323, 1)
# NuscData: 323 samples. Split: train.
#                    Augmentation Conf: {'final_dim': (448, 800), 'cams': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'], 'ncams': 6}
# indices (81, 1)
# NuscData: 81 samples. Split: val.
#                    Augmentation Conf: {'final_dim': (448, 800), 'cams': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'], 'ncams': 6}
# /usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
#   warnings.warn(
# data ready
# /usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
#   warnings.warn(
# /usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
#   warnings.warn(msg)
# Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
# 100% 171M/171M [00:11<00:00, 15.6MB/s]
# /usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
#   warnings.warn(msg)
# total_params 42048711
# reading ckpt from /content/simple_bev/checkpoints/8x5_5e-4_rgb12_22:43:46
# ...found checkpoint /content/simple_bev/checkpoints/8x5_5e-4_rgb12_22:43:46/model-000025000.pth
# /content/simple_bev/saverloader.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
#   checkpoint = torch.load(path, map_location=device)
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000001/20; rtime 0.01; itime 2.88 (2883.33 ms); loss 2.61083; iou_ev 55.4
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000002/20; rtime 0.03; itime 1.39 (2135.76 ms); loss 4.26774; iou_ev 55.4
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000003/20; rtime 0.05; itime 1.42 (1896.11 ms); loss 2.19172; iou_ev 51.5
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000004/20; rtime 0.04; itime 1.42 (1775.93 ms); loss 1.55265; iou_ev 51.6
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000005/20; rtime 0.04; itime 1.41 (1703.18 ms); loss 3.90031; iou_ev 51.3
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000006/20; rtime 0.03; itime 1.38 (1650.11 ms); loss 3.25109; iou_ev 51.5
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000007/20; rtime 0.02; itime 1.39 (1612.87 ms); loss 3.01142; iou_ev 51.1
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000008/20; rtime 0.02; itime 1.38 (1584.38 ms); loss 5.20864; iou_ev 50.5
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000009/20; rtime 0.02; itime 1.39 (1562.88 ms); loss 3.18122; iou_ev 50.1
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000010/20; rtime 0.02; itime 1.40 (1546.25 ms); loss 2.76588; iou_ev 49.6
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000011/20; rtime 0.02; itime 1.40 (1532.52 ms); loss 1.49694; iou_ev 49.4
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000012/20; rtime 0.04; itime 1.41 (1522.60 ms); loss 0.29988; iou_ev 49.5
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000013/20; rtime 0.06; itime 1.44 (1516.33 ms); loss -0.24335; iou_ev 50.0
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000014/20; rtime 0.04; itime 1.42 (1509.45 ms); loss 0.73640; iou_ev 50.0
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000015/20; rtime 0.05; itime 1.43 (1504.30 ms); loss 1.23849; iou_ev 50.2
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000016/20; rtime 0.03; itime 1.40 (1497.88 ms); loss 1.40093; iou_ev 50.6
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000017/20; rtime 0.03; itime 1.41 (1492.64 ms); loss 0.95781; iou_ev 50.9
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000018/20; rtime 0.03; itime 1.40 (1487.69 ms); loss 1.39125; iou_ev 51.1
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000019/20; rtime 0.03; itime 1.41 (1483.56 ms); loss 1.11533; iou_ev 51.7
# 8x5_5e-4_rgb12_22:43:46_4_eval_06:12:32; step 000020/20; rtime 0.03; itime 1.41 (1479.75 ms); loss 0.66789; iou_ev 52.0
# final  mean iou 52.015772263896906

In [13]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())


True
1


In [17]:
import torch
torch.cuda.empty_cache()  # Clears unused memory


In [21]:
!python eval_nuscenes.py \
       --batch_size=1 \
       --data_dir='/content/simple_bev/datasets/nuscenes' \
       --log_dir='logs_eval_nuscenes' \
       --init_dir='/content/simple_bev/checkpoints/8x5_5e-4_rad25_18:55:34' \
       --use_radar=True \
       --use_metaradar=True \
       --use_radar_filters=False \
       --res_scale=2 \
       --nsweeps=5 \
       --device_ids=[0]

model_name 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47
resolution: (448, 800)
loading nuscenes...
making parser...
indices (323, 1)
NuscData: 323 samples. Split: train.
                   Augmentation Conf: {'final_dim': (448, 800), 'cams': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'], 'ncams': 6}
indices (81, 1)
NuscData: 81 samples. Split: val.
                   Augmentation Conf: {'final_dim': (448, 800), 'cams': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'], 'ncams': 6}
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/f

In [15]:
# model_name 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47
# resolution: (448, 800)
# loading nuscenes...
# making parser...
# indices (323, 1)
# NuscData: 323 samples. Split: train.
#                    Augmentation Conf: {'final_dim': (448, 800), 'cams': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'], 'ncams': 6}
# indices (81, 1)
# NuscData: 81 samples. Split: val.
#                    Augmentation Conf: {'final_dim': (448, 800), 'cams': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'], 'ncams': 6}
# /usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
#   warnings.warn(
# data ready
# /usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
#   warnings.warn(
# /usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
#   warnings.warn(msg)
# /usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
#   warnings.warn(msg)
# total_params 42196167
# reading ckpt from /content/simple_bev/checkpoints/8x5_5e-4_rad25_18:55:34
# ...found checkpoint /content/simple_bev/checkpoints/8x5_5e-4_rad25_18:55:34/model-000025000.pth
# /content/simple_bev/saverloader.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
#   checkpoint = torch.load(path, map_location=device)
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000001/81; rtime 0.00; itime 1.06 (1058.97 ms); loss 0.71979; iou_ev 62.8
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000002/81; rtime 0.01; itime 0.43 (744.28 ms); loss 2.10723; iou_ev 61.1
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000003/81; rtime 0.01; itime 0.46 (651.11 ms); loss 2.48701; iou_ev 61.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000004/81; rtime 0.01; itime 0.42 (593.22 ms); loss 3.42402; iou_ev 62.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000005/81; rtime 0.03; itime 0.47 (569.11 ms); loss 3.45296; iou_ev 63.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000006/81; rtime 0.03; itime 0.47 (552.19 ms); loss 5.33240; iou_ev 62.5
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000007/81; rtime 0.03; itime 0.45 (537.40 ms); loss 4.84213; iou_ev 61.3
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000008/81; rtime 0.01; itime 0.43 (523.58 ms); loss 3.53004; iou_ev 61.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000009/81; rtime 0.04; itime 0.47 (517.69 ms); loss 2.93707; iou_ev 60.1
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000010/81; rtime 0.03; itime 0.46 (512.25 ms); loss 1.81340; iou_ev 59.8
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000011/81; rtime 0.03; itime 0.49 (509.87 ms); loss 2.51282; iou_ev 58.9
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000012/81; rtime 0.03; itime 0.48 (507.42 ms); loss 1.75047; iou_ev 58.2
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000013/81; rtime 0.09; itime 0.53 (509.28 ms); loss 0.97224; iou_ev 58.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000014/81; rtime 0.04; itime 0.50 (508.53 ms); loss 1.43313; iou_ev 58.3
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000015/81; rtime 0.08; itime 0.59 (514.16 ms); loss 2.22645; iou_ev 58.2
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000016/81; rtime 0.03; itime 0.49 (512.77 ms); loss 2.32269; iou_ev 57.7
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000017/81; rtime 0.07; itime 0.60 (518.11 ms); loss 1.98266; iou_ev 57.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000018/81; rtime 0.03; itime 0.52 (518.05 ms); loss 3.22442; iou_ev 57.2
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000019/81; rtime 0.03; itime 0.53 (518.61 ms); loss 3.96245; iou_ev 56.9
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000020/81; rtime 0.05; itime 0.50 (517.80 ms); loss 3.42187; iou_ev 56.8
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000021/81; rtime 0.02; itime 0.47 (515.64 ms); loss 4.27290; iou_ev 56.9
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000022/81; rtime 0.01; itime 0.44 (512.01 ms); loss 1.95910; iou_ev 57.1
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000023/81; rtime 0.01; itime 0.42 (508.15 ms); loss 1.82281; iou_ev 57.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000024/81; rtime 0.01; itime 0.42 (504.64 ms); loss 2.86079; iou_ev 57.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000025/81; rtime 0.01; itime 0.44 (501.96 ms); loss 3.13958; iou_ev 57.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000026/81; rtime 0.01; itime 0.43 (499.13 ms); loss 2.21563; iou_ev 56.8
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000027/81; rtime 0.01; itime 0.42 (496.19 ms); loss 2.69057; iou_ev 56.5
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000028/81; rtime 0.01; itime 0.43 (493.88 ms); loss 5.12379; iou_ev 56.1
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000029/81; rtime 0.01; itime 0.43 (491.51 ms); loss 4.60905; iou_ev 55.8
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000030/81; rtime 0.01; itime 0.43 (489.48 ms); loss 4.93626; iou_ev 55.7
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000031/81; rtime 0.01; itime 0.42 (487.37 ms); loss 4.68686; iou_ev 55.8
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000032/81; rtime 0.01; itime 0.42 (485.34 ms); loss 4.76390; iou_ev 55.6
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000033/81; rtime 0.01; itime 0.43 (483.55 ms); loss 4.68312; iou_ev 55.7
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000034/81; rtime 0.01; itime 0.43 (481.94 ms); loss 2.14782; iou_ev 55.7
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000035/81; rtime 0.01; itime 0.43 (480.45 ms); loss 1.76570; iou_ev 55.6
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000036/81; rtime 0.01; itime 0.43 (478.95 ms); loss 2.54162; iou_ev 55.5
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000037/81; rtime 0.01; itime 0.44 (477.76 ms); loss 1.96780; iou_ev 55.6
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000038/81; rtime 0.01; itime 0.43 (476.47 ms); loss 2.98115; iou_ev 55.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000039/81; rtime 0.01; itime 0.43 (475.27 ms); loss 2.26212; iou_ev 55.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000040/81; rtime 0.01; itime 0.43 (474.11 ms); loss 3.19870; iou_ev 55.3
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000041/81; rtime 0.01; itime 0.44 (473.23 ms); loss 2.94518; iou_ev 55.2
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000042/81; rtime 0.02; itime 0.45 (472.57 ms); loss -1.23203; iou_ev 55.2
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000043/81; rtime 0.01; itime 0.43 (471.61 ms); loss -0.41234; iou_ev 55.2
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000044/81; rtime 0.01; itime 0.43 (470.76 ms); loss -0.75311; iou_ev 55.3
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000045/81; rtime 0.03; itime 0.47 (470.80 ms); loss -1.11896; iou_ev 55.3
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000046/81; rtime 0.03; itime 0.48 (471.04 ms); loss 0.79001; iou_ev 55.3
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000047/81; rtime 0.08; itime 0.58 (473.28 ms); loss -0.96448; iou_ev 55.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000048/81; rtime 0.04; itime 0.51 (474.03 ms); loss -1.20484; iou_ev 55.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000049/81; rtime 0.05; itime 0.55 (475.49 ms); loss -0.07810; iou_ev 55.5
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000050/81; rtime 0.06; itime 0.54 (476.85 ms); loss -0.23125; iou_ev 55.6
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000051/81; rtime 0.04; itime 0.52 (477.61 ms); loss -0.92329; iou_ev 55.7
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000052/81; rtime 0.07; itime 0.55 (479.00 ms); loss -1.15746; iou_ev 55.8
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000053/81; rtime 0.06; itime 0.54 (480.21 ms); loss -0.17607; iou_ev 55.8
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000054/81; rtime 0.03; itime 0.48 (480.16 ms); loss -0.35572; iou_ev 55.9
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000055/81; rtime 0.04; itime 0.46 (479.71 ms); loss 0.38624; iou_ev 56.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000056/81; rtime 0.03; itime 0.46 (479.44 ms); loss 0.13775; iou_ev 56.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000057/81; rtime 0.01; itime 0.44 (478.74 ms); loss 0.77951; iou_ev 56.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000058/81; rtime 0.01; itime 0.43 (477.85 ms); loss 0.49129; iou_ev 56.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000059/81; rtime 0.01; itime 0.43 (476.97 ms); loss 1.73248; iou_ev 56.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000060/81; rtime 0.01; itime 0.42 (475.97 ms); loss 0.75143; iou_ev 56.1
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000061/81; rtime 0.01; itime 0.42 (475.06 ms); loss 1.68368; iou_ev 56.2
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000062/81; rtime 0.01; itime 0.43 (474.26 ms); loss 2.15206; iou_ev 56.1
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000063/81; rtime 0.03; itime 0.48 (474.32 ms); loss 2.33688; iou_ev 56.2
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000064/81; rtime 0.01; itime 0.43 (473.66 ms); loss 1.19391; iou_ev 56.2
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000065/81; rtime 0.01; itime 0.43 (472.91 ms); loss 1.90044; iou_ev 56.3
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000066/81; rtime 0.01; itime 0.42 (472.15 ms); loss 1.35696; iou_ev 56.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000067/81; rtime 0.01; itime 0.42 (471.44 ms); loss 1.50005; iou_ev 56.5
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000068/81; rtime 0.01; itime 0.43 (470.80 ms); loss 1.53225; iou_ev 56.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000069/81; rtime 0.01; itime 0.42 (470.12 ms); loss 1.03094; iou_ev 56.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000070/81; rtime 0.01; itime 0.42 (469.40 ms); loss 1.04826; iou_ev 56.4
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000071/81; rtime 0.01; itime 0.43 (468.80 ms); loss 0.61228; iou_ev 56.5
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000072/81; rtime 0.01; itime 0.43 (468.26 ms); loss 1.14685; iou_ev 56.5
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000073/81; rtime 0.01; itime 0.42 (467.66 ms); loss 0.69228; iou_ev 56.6
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000074/81; rtime 0.01; itime 0.42 (467.04 ms); loss 0.76994; iou_ev 56.7
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000075/81; rtime 0.01; itime 0.43 (466.49 ms); loss 0.29312; iou_ev 56.8
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000076/81; rtime 0.01; itime 0.43 (466.03 ms); loss 0.49912; iou_ev 56.9
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000077/81; rtime 0.01; itime 0.45 (465.78 ms); loss 0.80981; iou_ev 57.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000078/81; rtime 0.01; itime 0.45 (465.52 ms); loss 0.55996; iou_ev 57.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000079/81; rtime 0.01; itime 0.43 (465.12 ms); loss 0.33338; iou_ev 57.0
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000080/81; rtime 0.01; itime 0.43 (464.69 ms); loss 0.67832; iou_ev 57.1
# 8x5_5e-4_rad25_18:55:34_1_eval_06:18:47; step 000081/81; rtime 0.01; itime 0.44 (464.33 ms); loss 1.05691; iou_ev 57.1
# final  mean iou 57.120884073163325